In [1]:
%load_ext autoreload
%autoreload 2

# Recruitment Case Study

The following details the steps of the Recruitments case-study as described in the D3.3 deliverable.

## Step 0 - Setting up the working environment.

In [2]:
from flow import Network

## Step 1 - Create the basic structure of the information flow model.

**We can use the Network utility to create a simple linear information flow.** We use this to create some initial sites and channels describing our use case.

In [3]:
N = Network(
    site_ids=['(candidate)', 'Reg', 'Offered', 'Confirmed', 
              'Selected', 'Shortlist', 'Hired'], 
    channel_ids=['Candidate reg', 'REC search', 'Offer presented', 
                 'REC selection', 'REC interview', 'Client interview'])

In [4]:
# We can print it to look how it looks like.
N.plt()

## Step 2. Refine the model.

The initial structure was way too simple. We need more sites and channels to make it repesentative of the use case.

**We need some more sites to make the above simple picture more detailed.**

In [5]:
client, jp, cv, inp = N.sites('(client)', 'JP', 'CV', '(c. in person)', create=True)

**Create the missing channel**

In [6]:
N.channel(client, jp, id='REC create JP').label = 'Create JP'

**These are now the sites:**

In [7]:
N._sites

{'_s0': (candidate),
 '_s1': Reg,
 '_s2': Offered,
 '_s3': Confirmed,
 '_s4': Selected,
 '_s5': Shortlist,
 '_s6': Hired,
 '_s7': (client),
 '_s8': JP,
 '_s9': CV,
 '_s10': (c. in person)}

**We might wonder if the structure we have created thus far works.**

We can run *compute_order()*.

This will reveal that we have orphaned sites which are not yet connected.
If some channels were unreachable this would also be revealed.

In [8]:
N.compute_order()

**We see here that the new sites needs to be connected to the channels.**

In [9]:
N._channels

{'_c0': <Candidate reg: [(candidate)] --> {Reg}>,
 '_c6': <Create JP: [(client)] --> {JP}>,
 '_c1': <REC search: [Reg] --> {Offered}>,
 '_c2': <Offer presented: [Offered] --> {Confirmed}>,
 '_c3': <REC selection: [Confirmed] --> {Selected}>,
 '_c4': <REC interview: [Selected] --> {Shortlist}>,
 '_c5': <Client interview: [Shortlist] --> {Hired}>}

In [10]:
# The new sites needs to be connected
c1, c4, c5, c2 = N.channels('_c1', '_c4', '_c5', '_c2')
c1.add_input(jp)
c4.add_input(inp).add_input(cv)
c5.add_input(inp).add_input(cv)
c2.add_output(cv)

<Offer presented: [Offered] --> {Confirmed, CV}>

In [11]:
# Compute how the channels and site relates to each other. 
# This will both connect channels together in a graph and 
# detect problems like orphaned or unreachable sites or channels.
N.compute_order()

**Now the graph is without errors**

In [12]:
# Show the new computed version. It now presents the channels in information-flow order.
N.plt()

**Basic structure complete.**

Now this starts to look like the real recruitment process.

## Step 3 - Assign types

Here we assign types to information sites.
Types can be assigned and created manually or through the labels early on if we already have that information.

In [13]:
N.sites().levels

[(0, (candidate)),
 (0, (client)),
 (0, (c. in person)),
 (1, Reg),
 (1, JP),
 (2, Offered),
 (3, Confirmed),
 (3, CV),
 (4, Selected),
 (5, Shortlist),
 (6, Hired)]

In [14]:
N.channels().levels

[(0, <Candidate reg: [(candidate)] --> {Reg}>),
 (0, <Create JP: [(client)] --> {JP}>),
 (1, <REC search: [Reg, JP] --> {Offered}>),
 (2, <Offer presented: [Offered] --> {Confirmed, CV}>),
 (3, <REC selection: [Confirmed] --> {Selected}>),
 (4, <REC interview: [Selected, CV, (c. in person)] --> {Shortlist}>),
 (5, <Client interview: [Shortlist, CV, (c. in person)] --> {Hired}>)]

In [15]:
N2 = Network(site_ids=['Rec_Ext', 'Offered'], channel_ids=['AI Model Search'])

In [16]:
N2.plt()

In [17]:
s1, s2 = N2.sites('JP_Ext', '[Dataset]', create=True)

In [18]:
N2._channels

{'_c0': <AI Model Search: [Rec_Ext] --> {Offered}>}

In [19]:
N2.channel('_c0').add_input(s1).add_input(s2)

<AI Model Search: [Rec_Ext, JP_Ext, [Dataset]] --> {Offered}>

In [20]:
N2.plt()

In [21]:
print(N2.tex())

%_c0: [Rec_Ext, JP_Ext, [Dataset]] --> {Offered}

    \[
    \begin{tikzpicture}[node distance=1em, auto]
        % Define nodes
        \node (lab) at (-4, 0) {c0};
        \node (mr) at (10, 0) {};
        \node (ml) at (-5, 0) {};
        \node (r) at (0, 0) {};
        \node (l) at (3, 0) {};
            \node  (s0) [left=of r, yshift=-1.0em] {Rec_Ext};
\node  (s2) [left=of r, yshift=0.0em] {JP_Ext};
\node  (s3) [left=of r, yshift=1.0em] {[Dataset]};

            \node  (s1) [right=of l, yshift=0.0em] {Offered};

        % Draw arrow
        \draw[->] (r.east) -- (l.west) node[midway, above] {\textit{AI Model Search}};
    \end{tikzpicture}
    \]
    
